In [1]:
!which python

/Users/ron/Documents/github/LLMs-from-scratch-main/.venv/bin/python


In [2]:
import os
import urllib.request 

In [7]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [8]:
len(raw_text)

20479

In [9]:
import re
text = "Hello, world. This, is a test."
result = re.split(r'(\s)', text)
print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


In [10]:
result = re.split(r'([..]|\s)', text)
print(result)

['Hello,', ' ', 'world', '.', '', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [11]:
resullt = [item for item in result if item.strip()]
print(resullt)

['Hello,', 'world', '.', 'This,', 'is', 'a', 'test', '.']


In [12]:
text = "Hello, world. Is this-- a test?"

result = re.split(r'([,.:;?_!"()\']|--|\s)', text)
result = [item.strip() for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'Is', 'this', '--', 'a', 'test', '?']


In [13]:
result = re.split(r'([,.:;?_!"()\']|--|\s)', raw_text)
result = [item.strip() for item in result if item.strip()]
print(result)

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in', 'the', 'height', 'of', 'his', 'glory', ',', 'he', 'had', 'dropped', 'his', 'painting', ',', 'married', 'a', 'rich', 'widow', ',', 'and', 'established', 'himself', 'in', 'a', 'villa', 'on', 'the', 'Riviera', '.', '(', 'Though', 'I', 'rather', 'thought', 'it', 'would', 'have', 'been', 'Rome', 'or', 'Florence', '.', ')', '"', 'The', 'height', 'of', 'his', 'glory', '"', '--', 'that', 'was', 'what', 'the', 'women', 'called', 'it', '.', 'I', 'can', 'hear', 'Mrs', '.', 'Gideon', 'Thwing', '--', 'his', 'last', 'Chicago', 'sitter', '--', 'deploring', 'his', 'unaccountable', 'abdication', '.', '"', 'Of', 'course', 'it', "'", 's', 'going', 'to', 'send', 'the', 'value', 'of', 'my', 'picture', "'", 'way', 'up', ';', 'but', 'I', 'don', "'", 't', 'think', 'of', 'that', ',

In [14]:
preprocessed = result

In [15]:
len(preprocessed)

4690

## 2.3 Converting Tokens into Token IDs ##

In [16]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)
print(vocab_size)

1130


In [17]:
vocab = {token:integer for integer,token in enumerate(all_words)}

In [18]:
list(vocab.items())[:5]

[('!', 0), ('"', 1), ("'", 2), ('(', 3), (')', 4)]

In [19]:
class SimpleTokenizerV1():
    def __init__(self, vocab):
        self.str_to_int = {item:integer for integer,item in enumerate(vocab)}
        self.int_to_str = {integer:item for integer,item in enumerate(vocab)}
        
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        ids = [self.str_to_int[token] for token in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

In [20]:
tokenizer = SimpleTokenizerV1(vocab)
text1 = """\
"It's the last he painted, you know," 
Mrs. Gisburn said with pardonable pride."""


In [21]:
ids = tokenizer.encode(text1)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [22]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [60]:
tokenizer.decode(tokenizer.encode(text1))

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

## 2.4  Adding special  context tokens

In [23]:
text = "Hello, do you like tea--"

In [24]:
tokenizer = SimpleTokenizerV1(vocab)
ids = tokenizer.encode(text)

KeyError: 'Hello'

In [26]:
## 'Hello' is not in vocab

In [27]:
all_tokens = sorted(set(preprocessed))
all_tokens.extend(["<|endoftext|>","<|unk|>"])
vocab = {token:integer for integer,token in enumerate(all_tokens)}

In [28]:
len(vocab.items())

1132

In [29]:
class SimpleTokenizerV2():
    def __init__(self, vocab):
        self.str_to_int = {item:integer for integer,item in enumerate(vocab)}
        self.int_to_str = {integer:item for integer,item in enumerate(vocab)}
        
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]
        ids = [self.str_to_int[token] for token in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

In [30]:
tokenizer = SimpleTokenizerV2(vocab)
ids = tokenizer.encode(text)

In [31]:
ids

[1131, 5, 355, 1126, 628, 975, 6]

In [32]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea --'

## 2.5 Byte pair encoding 

In [33]:
import tiktoken 

In [34]:
tiktoken.__version__

'0.11.0'

In [35]:
tokenizer = tiktoken.get_encoding("gpt2")

In [36]:
tokenizer

<Encoding 'gpt2'>

In [37]:
tokenizer.encode("Hello world")

[15496, 995]

In [38]:
tokenizer.decode(tokenizer.encode("Hello world"))

'Hello world'

In [39]:
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [79]:
text

'Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.'

## 2.6 Data Sampling with Sliding Window 

In [40]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [41]:
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [42]:
enc_sample = enc_text[50:]

In [43]:
len(enc_sample)

5095

In [44]:
context_size = 4
x = enc_sample[:context_size]
y = enc_sample[1:context_size+1]

In [45]:
print(f"x: {x}")
print(f"y:     {y}")

x: [290, 4920, 2241, 287]
y:     [4920, 2241, 287, 257]


In [46]:
y

[4920, 2241, 287, 257]

In [47]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(context, "---->", desired)

[290] ----> 4920
[290, 4920] ----> 2241
[290, 4920, 2241] ----> 287
[290, 4920, 2241, 287] ----> 257


In [48]:
for i in range(1, context_size+1):
    context = enc_sample[:i]
    desired = enc_sample[i]

    print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

 and ---->  established
 and established ---->  himself
 and established himself ---->  in
 and established himself in ---->  a


In [95]:
import torch

In [96]:
 torch.__version__

'2.8.0'

In [54]:
from torch.utils.data import Dataset, DataLoader
import torch

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})
        assert len(token_ids) > max_length, "Number of tokenized inputs must at least be equal to max_length+1"

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i+max_length]
            target_chunk = token_ids[i+1: i+1+max_length]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))


    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [55]:
def create_dataloader_v1(txt, batch_size=1, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)
    

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

In [56]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()

In [61]:
dataloader = create_dataloader_v1(raw_text, batch_size=1, max_length=4, stride=4, shuffle=False, drop_last=True)

In [62]:
data_iter = iter(dataloader)
first_batch = next(data_iter)

In [63]:
print(first_batch)

[tensor([[  40,  367, 2885, 1464]]), tensor([[ 367, 2885, 1464, 1807]])]


In [64]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[1807, 3619,  402,  271]]), tensor([[ 3619,   402,   271, 10899]])]


In [138]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

Targets:
 tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])


## 2.7 Creating Token Embedding ##

In [65]:
input_ids =  torch.tensor([2, 3, 5, 1])

In [66]:
dir(tokenizer)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_core_bpe',
 '_encode_bytes',
 '_encode_only_native_bpe',
 '_encode_single_piece',
 '_mergeable_ranks',
 '_pat_str',
 '_special_tokens',
 'decode',
 'decode_batch',
 'decode_bytes',
 'decode_bytes_batch',
 'decode_single_token_bytes',
 'decode_tokens_bytes',
 'decode_with_offsets',
 'encode',
 'encode_batch',
 'encode_ordinary',
 'encode_ordinary_batch',
 'encode_single_token',
 'encode_to_numpy',
 'encode_with_unstable',
 'eot_token',
 'is_special_token',
 'max_token_value',
 'n_vocab',
 'name',
 'special_tokens_set',
 'token_byte_values']

In [67]:
tokenizer.n_vocab

50257

In [68]:
tokenizer.special_tokens_set

{'<|endoftext|>'}

In [69]:
tokenizer.token_byte_values

<bound method Encoding.token_byte_values of <Encoding 'gpt2'>>

In [70]:
vocab_size = 6
output_dim =3
torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [71]:
embedding_layer.weight

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)

In [72]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


In [73]:
print(embedding_layer(torch.tensor([2])))

tensor([[ 1.2753, -0.2010, -0.1606]], grad_fn=<EmbeddingBackward0>)


In [74]:
embedding_layer(input_ids)

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)

## Encoding Word Positions

In [75]:
vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [76]:
max_length=4
dataloader = create_dataloader_v1(
    raw_text, batch_size=8, max_length=max_length, stride=max_length, shuffle=False
)
data_iter = iter(dataloader)
inputs, targets = next(data_iter)

In [77]:
print("Token IDs: \n", inputs)
print("\n Inputs shape \n", inputs.shape)

Token IDs: 
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])

 Inputs shape 
 torch.Size([8, 4])


In [78]:
token_embeddings = token_embedding_layer(inputs)
token_embeddings.shape

torch.Size([8, 4, 256])

In [170]:
#token_embeddings[0,0]

In [79]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)

In [80]:
torch.arange(max_length)

tensor([0, 1, 2, 3])

In [81]:
pos_embedding_layer.weight

Parameter containing:
tensor([[ 1.7375, -0.5620, -0.6303,  ..., -0.2277,  1.5748,  1.0345],
        [ 1.6423, -0.7201,  0.2062,  ...,  0.4118,  0.1498, -0.4628],
        [-0.4651, -0.7757,  0.5806,  ...,  1.4335, -0.4963,  0.8579],
        [-0.6754, -0.4628,  1.4323,  ...,  0.8139, -0.7088,  0.4827]],
       requires_grad=True)

In [82]:
pos_embeddings = pos_embedding_layer(torch.arange(max_length))
pos_embeddings.shape

torch.Size([4, 256])

In [83]:
token_embeddings[0]

tensor([[ 0.4913,  1.1239,  1.4588,  ..., -0.3995, -1.8735, -0.1445],
        [ 0.4481,  0.2536, -0.2655,  ...,  0.4997, -1.1991, -1.1844],
        [-0.2507, -0.0546,  0.6687,  ...,  0.9618,  2.3737, -0.0528],
        [ 0.9457,  0.8657,  1.6191,  ..., -0.4544, -0.7460,  0.3483]],
       grad_fn=<SelectBackward0>)

In [84]:
token_embeddings[0].shape

torch.Size([4, 256])

In [175]:
# Braodcasting
token_embeddings[0]+pos_embeddings

tensor([[-0.1390,  0.6390,  1.3222,  ...,  0.6350, -2.3747,  0.9599],
        [ 0.6543,  0.8614,  0.4532,  ...,  0.0369, -1.4310,  0.0136],
        [ 0.3299, -1.4393,  0.9953,  ...,  1.8197,  2.8795,  0.9715],
        [ 2.3781,  1.0874,  2.4790,  ...,  0.0283,  0.0999,  1.6521]],
       grad_fn=<AddBackward0>)

In [176]:
token_embeddings+pos_embeddings

tensor([[[-0.1390,  0.6390,  1.3222,  ...,  0.6350, -2.3747,  0.9599],
         [ 0.6543,  0.8614,  0.4532,  ...,  0.0369, -1.4310,  0.0136],
         [ 0.3299, -1.4393,  0.9953,  ...,  1.8197,  2.8795,  0.9715],
         [ 2.3781,  1.0874,  2.4790,  ...,  0.0283,  0.0999,  1.6521]],

        [[ 0.9158,  1.2520, -0.9214,  ...,  0.9341,  0.3572,  0.7624],
         [-1.6560,  0.4164,  0.3375,  ...,  0.6591, -0.5815,  1.8070],
         [ 2.5653, -2.0329,  0.1851,  ...,  0.4738, -0.4297,  2.4721],
         [ 2.3971,  1.5190, -0.7608,  ...,  1.6290,  2.4256,  1.7007]],

        [[-1.4016,  0.1724,  0.0297,  ...,  0.2302, -0.4470,  1.8471],
         [ 1.0107,  1.1125,  2.0109,  ...,  1.0020,  0.1778,  1.5185],
         [ 0.6601, -3.1482,  0.9016,  ...,  3.0402,  2.3289,  0.6608],
         [ 1.8590,  0.1569,  1.4285,  ..., -0.0382,  2.1524,  2.1511]],

        ...,

        [[-2.2458,  0.4762, -2.7803,  ...,  0.0701,  0.5877,  2.7428],
         [-0.1923, -0.3157, -0.5976,  ..., -1.6210, -1.36

In [177]:
input_embeddings = token_embeddings+pos_embeddings
input_embeddings.shape

torch.Size([8, 4, 256])